In [1]:
import sys
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
from scipy import stats
from scipy.stats import rankdata
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
np.random.seed(1)

In [2]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

**One-hot encoding**

In [3]:
 def one_hot_encode(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df = pd.concat([df, pd.get_dummies(df['gender'], prefix='gender')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['race'], prefix='race')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['marital'], prefix='marital')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['workclass'], prefix='workclass')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['relationship'], prefix='relationship')],axis=1)
    df = pd.concat([df, pd.get_dummies(df['occupation'], prefix='occupation')],axis=1)

    df = df.drop(columns=['workclass', 'gender', 'fnlwgt', 'education', 'occupation', \
                      'relationship', 'marital', 'race', 'country', 'capgain', \
                      'caploss'])
    return df

# one-hot encoding (for regression mdoels)
df_train = one_hot_encode(df_train)
df_test = one_hot_encode(df_test)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

**Protected, privileged**

In [4]:
# protected: 'gender_Female'=1
# privileged: 'gender_Male'=1

**Parametric Model**

In [5]:
X_train = df_train.drop(columns='income')
y_train = df_train['income']

X_test = df_test.drop(columns='income')
y_test = df_test['income']

X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=300)

**Compute fairness metric**

In [22]:
def computeFairness(y_pred, X_test, y_test, metric): 
    fairnessMetric = 0
    protected_idx = X_test[X_test['gender_Female']==1].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender_Male']==1].index
    numPrivileged = len(privileged_idx)
    
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    # statistical parity difference
    statistical_parity = p_protected - p_privileged
    
    # equalized odds 
    # true positive rate parity
    # P(Y=1 | Y=1, G=0)- P(Y=1 | Y=1, G=1)
    true_positive_protected = 0
    actual_positive_protected = 0
    for i in range(len(protected_idx)):
        if (y_test[protected_idx[i]] == 1):
            actual_positive_protected += 1
            if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
                true_positive_protected += 1
    tpr_protected = true_positive_protected/actual_positive_protected
    
    true_positive_privileged = 0
    actual_positive_privileged = 0
    for i in range(len(privileged_idx)):
        if (y_test[privileged_idx[i]] == 1):
            actual_positive_privileged += 1
            if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
                true_positive_privileged += 1
    tpr_privileged = true_positive_privileged/actual_positive_privileged
    
    tpr_parity = tpr_protected - tpr_privileged
    
    if (metric == 0):
        fairnessMetric = statistical_parity
    elif (metric == 1):
        fairnessMetric = tpr_parity
    
    # false positive rate parity
    
    # predictive parity
    
    return fairnessMetric

**Influence of points computed using ground truth**

In [7]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig, y_test, 0)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig, y_test, 0) - spd_0
        delta_spd.append(delta_spd_i)
    
    return delta_spd

**Loss function** (Log loss for logistic regression)

In [8]:
def logistic_loss(y_true, y_pred):
    loss = 0
    for i in range(len(y_true)):
        if (y_pred[i][1] != 0 and y_pred[i][0] != 0):
            loss += - y_true[i] * math.log(y_pred[i][1]) - (1 - y_true[i]) * math.log(y_pred[i][0])
    loss /= len(y_true)
    return loss

**Compute Accuracy** 

In [9]:
def computeAccuracy(y_true, y_pred):
    accuracy = 0
    for i in range(len(y_true)):
        idx = y_true[i]
        if (y_pred[i][idx] > y_pred[i][1 - idx]):
            accuracy += 1
#         accuracy += y_pred[i][idx]
    accuracy /= len(y_true)
    return accuracy

**First-order derivative of loss function at z with respect to model parameters**

(Pre-computed for all training points)

In [10]:
def del_L_del_theta_i(num_params, y_true, x, y_pred):
#     del_L_del_theta = np.ones((num_params, 1)) * ((1 - y_true) * y_pred[1] - y_true * y_pred[0])
    del_L_del_theta = np.ones((num_params, 1)) * (- y_true + y_pred[1])
    for j in range(1, num_params):
            del_L_del_theta[j] *=  x[j-1]
    return del_L_del_theta

**Hessian: Second-order partial derivative of loss function with respect to model parameters**

(Pre-computed for all training points)

In [11]:
def hessian_one_point(num_params, x, y_pred):
    H = np.ones((num_params, num_params)) * (y_pred[0] * y_pred[1])
    for i in range(1, num_params):
        for j in range(i+1):
            if j == 0:
                H[i][j] *= x[i-1]
            else:
                H[i][j] *= x[i-1] * x[j-1] 
    i_lower = np.tril_indices(num_params, -1)
    H.T[i_lower] = H[i_lower]     
    return H

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [12]:
def del_f_del_theta_i(num_params, x, y_pred):
    del_f_del_theta = np.ones((num_params, 1)) * (y_pred[0] * y_pred[1])
    for j in range(1, num_params):
            del_f_del_theta[j] *=  x[j-1]
    return del_f_del_theta

**Computing $v=\nabla($Statistical parity difference$)$**

In [13]:
# Return v = del(SPD)/del(theta)
def del_spd_del_theta(num_params, X_test_orig, X_test, y_pred):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    numProtected = X_test_orig['gender_Female'].sum()
    numPrivileged = X_test_orig['gender_Male'].sum()
    for i in range(len(X_test)):
        del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[i])
        if X_test_orig.iloc[i]['gender_Male'] == 1: #privileged
            del_f_privileged = np.add(del_f_privileged, del_f_i)
        elif X_test_orig.iloc[i]['gender_Female'] == 1:
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_privileged /= numPrivileged
    del_f_protected /= numProtected
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($TPR parity difference$)$**

In [13]:
# Return v = del(TPR_parity)/del(theta)
def del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test[X_test['gender_Female']==1].index
    privileged_idx = X_test[X_test['gender_Male']==1].index
    
    p_y_1_protected = 0
    p_y_1_privileged = 0
    
    true_positive_protected = 0
    actual_positive_protected = 0
    for i in range(len(protected_idx)):
        if (y_test[protected_idx[i]] == 1):
            actual_positive_protected += 1
            if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
                true_positive_protected += 1
    tpr_protected = true_positive_protected/actual_positive_protected
    
    num_positive_privileged = 0
    num_positive_protected = 0
    for i in range(len(X_test)):
        del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[i])
        if X_test_orig.iloc[i]['gender_Male'] == 1: #privileged
            del_f_privileged = np.add(del_f_privileged, del_f_i)
        elif X_test_orig.iloc[i]['gender_Female'] == 1:
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_privileged =
    del_f_protected =
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [14]:
# Uniformly sample t points from training data 
def hessian_vector_product(num_params, n, size, v, hessian_all_points):
    if (size > n):
        size = n
    sample = random.sample(range(n), size)
    hinv_v = copy.deepcopy(v)
    for idx in range(size):
        i = sample[idx]
        hessian_i = hessian_all_points[i]
        hinv_v = np.matmul(np.subtract(np.identity(num_params), hessian_i), hinv_v)
        hinv_v = np.add(hinv_v, v)
    return hinv_v

**First-order influence computation**

In [15]:
def first_order_influence(del_L_del_theta, hinv_v, n):
    infs = []
    for i in range(n):
        inf = -np.dot(del_L_del_theta[i].transpose(), hinv_v)
        inf *= -1/n
        infs.append(inf[0][0].tolist())
    return infs

**Second-order Influence function computation**

(For any group of points U)

In [16]:
def second_order_influence(X_train, v1, U, size, del_L_del_theta, hessian_all_points):
    u = len(U) 
    s = len(X_train)
    p = u/s
    c1 = (1 - 2*p)/(s * (1-p)**2)
    c2 = 1/((s * (1-p))**2)
    num_params = len(v1)
    
    del_L_del_theta_hinv = np.zeros((num_params, 1))
    del_L_del_theta_sum = np.zeros((num_params, 1))
    hessian_all = np.zeros((num_params, num_params))
    for i in range(u):
        idx = U[i]
        del_L_del_theta_hinv = np.add(del_L_del_theta_hinv, hessian_vector_product(num_params, s, size, del_L_del_theta[idx], hessian_all_points))
        hessian_all = np.add(hessian_all, hessian_all_points[idx])
        del_L_del_theta_sum = np.add(del_L_del_theta_sum, del_L_del_theta[idx])
    
    term1 = c1 * del_L_del_theta_sum
    term2 = c2 * np.dot(hessian_all, del_L_del_theta_hinv)

    I = np.dot(v1.transpose(), (term1 + term2))
    return (I*(-1)) # multiplied by -1 because removing these points

**Metrics: Initial state**

In [23]:
threshold = 0.0001
clf.fit(X_train, y_train)
num_params = len(clf.coef_.transpose()) + 1 #weights and intercept; params: clf.coef_, clf.intercept_
y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)
    
spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1)
print("Initial fairness: ", spd_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial fairness:  -0.16734289130157864
Initial loss:  0.360972684923813
Initial accuracy:  0.8289508632138114


**Pre-compute: (1) Hessian (2) del_L_del_theta for each training data point**

In [23]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    del_L_del_theta.insert(i, del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred_train[i]))

hessian_all_points = []
for i in range(len(X_train)):
    hessian_all_points.insert(i, hessian_one_point(num_params, X_train[i], y_pred_train[i])
                              /len(X_train))

**Compute: (1) First-order influence, (2) Ground truth influence of each training data point**

In [19]:
# Ground truth influence
# spdgt = ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test)
# with open('delta_spd_ground_truth_v0.txt', 'w') as filehandle:
#     for listitem in delta_spd:
#         filehandle.write('%s\n' % listitem)
gt_spd = pd.read_csv('delta_spd_ground_truth_v0.txt', names=["Values"], sep=",")
gt_spd = gt_spd.values.tolist()
spdgt=[]
for i in range(len(gt_spd)):
    spdgt.append(gt_spd[i][0])
sort_index = np.argsort(spdgt)[::-1][:len(spdgt)]

In [20]:
size_hvp = int(len(X_train) * .001)
# Hessian vector product H^{-1}v, v = del_fairness
v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)
# v = del_L_del_theta[3]
hinv_v = hessian_vector_product(num_params, len(X_train), size_hvp, v1, hessian_all_points)

infs_1 = first_order_influence(del_L_del_theta, hinv_v, len(X_train))
print("Spearman rank correlation between 1st order inf and ground truth inf: ", 
      stats.spearmanr(spdgt, infs_1)[0])
print("Pearson correlation coefficient between 1st order inf and ground truth inf: ", 
      stats.pearsonr(spdgt, infs_1)[0])

Spearman rank correlation between 1st order inf and ground truth inf:  0.9222355399390547
Pearson correlation coefficient between 1st order inf and ground truth inf:  0.8903340052474299


**Space Partitioner for reducing bias**

In [19]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

def preprocess(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df = df.drop(columns=['fnlwgt', 'education.num', 'country', 'capgain', 'caploss'])
    return df

df_train = preprocess(df_train)
df_test = preprocess(df_test)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

X_train_ = df_train.drop(columns='income')
y_train_ = df_train['income']

X_test_ = df_test.drop(columns='income')
y_test_ = df_test['income']

In [20]:
def computeFairness(y_pred, X_test): 
    protected_idx = X_test[X_test['gender']=='Female'].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender']=='Male'].index
    numPrivileged = len(privileged_idx)
    
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    spd = p_protected - p_privileged
    return spd

def getInfluenceOfSet(indices, f, X_train, y_train, X_test, X_test_, method): 
    del_f = 0
    if (method == 1):
        X = X_train.drop(index=indices, inplace=False)
        y = y_train.drop(index=indices, inplace=False)
        if len(y.unique()) < 2:
            return 0
        clf.fit(X, y)
        y_pred = clf.predict_proba(X_test)
        del_f = computeFairness(y_pred, X_test_)
#     elif (method == 2):
#         for i in range(len(indices)):
#             del_f += infs_1[indices[i]]
#     elif (method == 3):
#         del_f = second_order_influence(X_df.to_numpy(), v1, indices, size_hvp, del_L_del_theta, hessian_all_points)
#     del_f = del_f * 100/f
    return  round(del_f, 3)

def getSplitVal(infs):
    return (np.argmax(np.asarray(infs)))

def getSplitAttribute(cols, cols_continuous, X_train, y_train, X_test, X_train_, X_test_, method):
    splitCol, numRows, score = None, len(X_train_), np.Inf
    infs = []
    vals = []
    idxs = []
    for i in range(len(cols)):
        col = cols[i]
        infs_i = []
        vals_i = []
        idxs_i = []
        if col not in cols_continuous:
            colVals = X_train_[col].unique()
            for val in colVals:
                idx = X_train_[X_train_[col] == val].index
                idxs_i.append(len(X_train)-len(idx))
                infs_i.append(getInfluenceOfSet(idx, spd_0, X_train, y_train, X_test, X_test_, method))
                vals_i.append(val)
                ix = getSplitVal(infs_i)
                if (abs(infs_i[ix]) < abs(spd_0)) and (abs(infs_i[ix]) < score):
                    print("Column passed: ", col)
                    print("Val: ", val)
                    splitCol = col
                    splitIdx = i
                    score = abs(infs_i[ix])
        infs.append(infs_i)
        vals.append(vals_i)
        idxs.append(idxs_i)
    return {'splitCol':splitCol, 'numRows':numRows, 
            'infs': infs[splitIdx], 'vals': vals[splitIdx], 'idxs': idxs[splitIdx]}
#         else:
#         vals = X_train_orig[col].unique()
#         vals.sort()
#         mid = []
#         for i in range(len(vals) - 1):
#             mid.append(np.mean(vals[i:i+2]))
#         for val in mid:
# #             print(val)
#             idxLeft = X_train_orig[X_train_orig[col] <= val].index
#             idxRight = X_train_orig[X_train_orig[col] > val].index
#             infLeft = getInfluenceOfSet(idxLeft, spd_0, X_df, y_df, X_test_df, X_test_orig, method)
#             infRight = getInfluenceOfSet(idxRight, spd_0, X_df, y_df, X_test_df, X_test_orig, method)
#             gain = getSplitGain(infLeft, infRight)
# #             if abs(gain) > abs(score):
#             if gain > score:
#                 print("Column passed: ", col)
#                 print("Gain: ", gain)
#                 splitCol, splitVal, score = col, val, gain 
#                 left, right = idxLeft, idxRight
#                 if method==1:
#                     leftInf, rightInf = infLeft, infRight
#                 else:
#                     leftInf = getInfluenceOfSet(idxLeft, spd_0, X_df, y_df, X_test_df, X_test_orig, 1)
#                     rightInf = getInfluenceOfSet(idxRight, spd_0, X_df, y_df, X_test_df, X_test_orig, 1)
#                 count = len(X_train_orig)    

def partition(node, maxDepth, minSize, depth, cols, cols_continuous, 
              X_train_, y_train_, X_train, X_test_, X_test, method):
    print("Depth: ", depth)
    if depth >= maxDepth or node['numRows'] < minSize:
        node['children'] = None
        return
    col = node['splitCol']
    if col not in cols_continuous:
        vals = X_train_[col].unique()
        child = [None] * len(vals)
        for i in range(len(vals)):
            idx = X_train_[X_train_[col] == vals[i]].index 
            X = X_train.drop(index=idx, inplace=False)
            y = y_train.drop(index=idx, inplace=False)
            X_ = X_train_.drop(index=idx, inplace=False)
            if len(X) < minSize:
                node['children'] = None
            else:
                cols_ = copy.deepcopy(cols)
                cols_.remove(col)
                child[i] = getSplitAttribute(cols_, cols_continuous,
                                             X, y, X_test, X_, X_test_, method)
                child[i]['col'] = col
                child[i]['val'] = vals[i]
                partition(child[i], maxDepth, minSize, depth + 1, cols_, cols_continuous, 
                  X_, y, X, X_test_, X_test, method)
    node['children'] = child

def buildTree(X_train_, X_train, maxDepth, minSize, method):
    cols = copy.deepcopy(X_train_.columns).tolist()
    cols_continuous = ['age', 'hours']
    X_train = pd.DataFrame(data=X_train, columns=X_train_orig.columns)
    cols = list(set(cols) - set(cols_continuous))
    root = getSplitAttribute(cols, cols_continuous,
                             X_train, y_train, X_test, X_train_, X_test_, method)
    print(root)
    partition(root, maxDepth, minSize, 1, cols, cols_continuous,
              X_train_, y_train_, X_train, X_test_, X_test, method)
    return root

method = 1
dtree = buildTree(X_train_, X_train, 2, 20, method)


Column passed:  occupation
Val:  Adm-clerical
Column passed:  occupation
Val:  Exec-managerial
Column passed:  gender
Val:  Male
Column passed:  gender
Val:  Female
Column passed:  marital
Val:  Married-civ-spouse
{'splitCol': 'marital', 'numRows': 30162, 'infs': [-0.208, -0.028, -0.189, -0.201, -0.199, -0.201, -0.197], 'vals': ['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated', 'Married-AF-spouse', 'Widowed'], 'idxs': [20436, 16097, 25948, 29792, 29223, 30141, 29335]}
Depth:  1
Column passed:  gender
Val:  Male
Depth:  2
Column passed:  occupation
Val:  Adm-clerical
Depth:  2
Column passed:  occupation
Val:  Adm-clerical
Column passed:  occupation
Val:  Exec-managerial
Column passed:  gender
Val:  Male
Depth:  2
Column passed:  occupation
Val:  Adm-clerical
Column passed:  occupation
Val:  Exec-managerial
Column passed:  gender
Val:  Male
Depth:  2
Column passed:  occupation
Val:  Adm-clerical
Column passed:  occupation
Val:  Exec-managerial
Colum

In [21]:
dtree

{'splitCol': 'marital',
 'numRows': 30162,
 'infs': [-0.208, -0.028, -0.189, -0.201, -0.199, -0.201, -0.197],
 'vals': ['Never-married',
  'Married-civ-spouse',
  'Divorced',
  'Married-spouse-absent',
  'Separated',
  'Married-AF-spouse',
  'Widowed'],
 'idxs': [20436, 16097, 25948, 29792, 29223, 30141, 29335],
 'children': [{'splitCol': 'gender',
   'numRows': 20436,
   'infs': [-0.055, -0.093],
   'vals': ['Male', 'Female'],
   'idxs': [5470, 14966],
   'col': 'marital',
   'val': 'Never-married',
   'children': None},
  {'splitCol': 'occupation',
   'numRows': 16097,
   'infs': [-0.009,
    -0.029,
    -0.041,
    -0.012,
    -0.02,
    -0.013,
    -0.029,
    -0.025,
    -0.026,
    -0.026,
    -0.041,
    -0.042,
    -0.027,
    -0.028],
   'vals': ['Adm-clerical',
    'Handlers-cleaners',
    'Other-service',
    'Prof-specialty',
    'Sales',
    'Farming-fishing',
    'Machine-op-inspct',
    'Exec-managerial',
    'Tech-support',
    'Craft-repair',
    'Protective-serv',
   

**Checking ground truth, first-order and second-order influences for a set**

In [23]:
predicates = ['marital_Married-civ-spouse', 'occupation_Armed-Forces']
# predicates = ['marital_Widowed']
# predicates = ['gender_Male']

idx = X_train_orig[(X_train_orig[predicates[0]] == 1)
                   & (X_train_orig[predicates[1]] == 1) 
#                    & (X_train_orig[predicates[2]] == 1)
#                    & (X_train_orig[predicates[3]] == 1)
                  ].index 
print(len(idx))
X = np.delete(X_train, idx, 0)
y = y_train.drop(index=idx, inplace=False)
clf.fit(X, y)
y_pred_test = clf.predict_proba(X_test)
print("Ground truth influence: ", computeFairness(y_pred_test, X_test_, y_test, 0))
print(spd_0)

3
Ground truth influence:  -0.20043790718523896
-0.20059371090978573
